In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # visualization
import matplotlib.pyplot as plt
import random
from sklearn.metrics import accuracy_score, log_loss
from scipy import optimize
from keras import losses
from keras import backend as K
import tensorflow as tf

%matplotlib notebook

Using TensorFlow backend.


# Preprocessing

In [2]:
data = pd.read_csv('winequality-white.csv',sep=';')
X = data.values[:,0:-1]
Y_first = data.values[:,-1]
Y = []
out = np.zeros(shape=[7])
for i in range(np.shape(Y_first)[0]):
    out[int(Y_first[i])-3]=1
    Y.append(out)
    out = np.zeros(shape=[7])
Y = np.array(Y)
# min_max_scaler = preprocessing.MinMaxScaler()
# X_scaled = min_max_scaler.fit_transform(X)
# df_normalized = pd.DataFrame(X_scaled)

r,c = np.shape(X)

total_samples = list(np.arange(0,r,1))
test_samples = random.sample(total_samples,int(np.round(len(total_samples)*0.2)))
train_samples = list(set(total_samples) - set(test_samples))

# Splitting Data

In [3]:
trainX = X[train_samples]
trainX_scaled = (trainX-np.min(trainX))/(np.max(trainX)-np.min(trainX))
trainY = Y[train_samples]
trainY_comp = Y_first[train_samples]

testX = X[test_samples]
testX_scaled = (testX-np.min(testX))/(np.max(testX)-np.min(testX))
testY = Y[test_samples]
testY_comp = Y_first[test_samples]
print(len(trainX[0]))
print(len(trainY[0]))

11
7


In [4]:
def logloss(true_label, predicted, eps=1e-15):
#   p = np.clip(predicted, eps, 1 - eps)
#   if true_label == 1:
#     return -np.log(p)
#   else:
#     return 0
    epsilon_ = _to_tensor(epsilon(), output.dtype.base_dtype)
    output = clip_ops.clip_by_value(output, epsilon_, 1. - epsilon_)
    return -math_ops.reduce_sum(target * math_ops.log(output), axis)

In [5]:
def de(fobj, bounds, mut=0.8, crossp=0.7, popsize=50, its=1000):
    dimensions = len(bounds)
    pop = np.random.rand(popsize, dimensions)
    min_b, max_b = np.asarray(bounds).T
    diff = np.fabs(min_b - max_b)
    pop_denorm = min_b + pop * diff
    fitness = np.asarray([fobj(ind) for ind in pop_denorm])
    best_idx = np.argmin(fitness)
    best = pop_denorm[best_idx]
    for i in range(its):
        print(i)
        for j in range(popsize):
            idxs = [idx for idx in range(popsize) if idx != j]
            a, b, c = pop[np.random.choice(idxs, 3, replace = False)]
            mutant = np.clip(a + mut * (b - c), 0, 1)
            cross_points = np.random.rand(dimensions) < crossp
            if not np.any(cross_points):
                cross_points[np.random.randint(0, dimensions)] = True
            trial = np.where(cross_points, mutant, pop[j])
            trial_denorm = min_b + trial * diff
            f = fobj(trial_denorm)
            if f < fitness[j]:
                fitness[j] = f
                pop[j] = trial
                if f < fitness[best_idx]:
                    best_idx = j
                    best = trial_denorm
                    print(fitness[best_idx])
        yield best, fitness[best_idx]

#  Neural Network

In [16]:
def nn_objective(x):
    num_inputs = 11
    num_outputs = 7

    hidden_layer_neurons = 40
    
    weights_all = num_inputs*hidden_layer_neurons + hidden_layer_neurons*num_outputs
    bias_all = hidden_layer_neurons+num_outputs
    iw_size = num_inputs*hidden_layer_neurons
    nn_weights = x[:weights_all]
    bias_all = x[weights_all:]
    iw = nn_weights[0:iw_size]
    ow = nn_weights[iw_size:]
    b1 = bias_all[:hidden_layer_neurons]
    b2 = bias_all[hidden_layer_neurons:]
    w1 = np.reshape(iw,(num_inputs, hidden_layer_neurons))
    w2 = np.reshape(ow,(hidden_layer_neurons, num_outputs))
#     l1 = 1/(1+np.exp(-(np.dot(trainX,w1)+b1)))
    l1 = np.maximum(np.dot(trainX, w1)+b1, 0)
    x_i = np.dot(l1, w2)+b2
    l2 = np.array(np.exp(x_i)/ np.exp(x_i).sum(axis=1)[:,None])  
#     l2 = np.array(np.exp(x_i)/ sum(np.exp(x_i)))
#     l2_act = np.argmax(l2,axis=1)+3
#     predict_out = l2_act.flatten()
#     er = 1-accuracy_score(trainY_comp,predict_out)
#     er_t = []
#     er_i = 0
#     for i in range(np.shape(trainY)[0]):
#         for j in range(np.shape(trainY)[1]):
#             er_i += logloss(np.array(trainY[i,j]),np.array(l2[i,j]))
#         #er_t.append(er_i)
#     er = np.mean(er_i)
    output = np.clip(l2, 1e-15, 1. - 1e-15)
    er= -np.sum(trainY * np.log(l2))
    return er



In [17]:
num_inputs = 11
num_outputs = 7

hidden_layer_neurons = 40

weights_all = num_inputs*hidden_layer_neurons + hidden_layer_neurons*num_outputs
bias_all = hidden_layer_neurons+num_outputs
#bias_all = 0

tw = weights_all+bias_all 
w0 = 2*np.random.random((1,tw))-2

b = [(-0.1,0.1)]
for i in range(len(w0[0])-1):
    b.append((-0.1,0.1))
#res = list(de(nn_objective,b))
res = optimize.basinhopping(nn_objective, w0[0], niter=10000,disp=True)


basinhopping step 0: f 5050.49
basinhopping step 1: f 5050.49 trial_f 5050.49 accepted 1  lowest_f 5050.49
found new global minimum on step 1 with function value 5050.49
basinhopping step 2: f 5050.49 trial_f 5050.49 accepted 1  lowest_f 5050.49
basinhopping step 3: f 5050.49 trial_f 5050.49 accepted 1  lowest_f 5050.49
found new global minimum on step 3 with function value 5050.49
basinhopping step 4: f 5050.49 trial_f 5050.49 accepted 1  lowest_f 5050.49
basinhopping step 5: f 5050.49 trial_f 5050.49 accepted 1  lowest_f 5050.49
found new global minimum on step 5 with function value 5050.49
basinhopping step 6: f 5050.49 trial_f 5050.49 accepted 1  lowest_f 5050.49
found new global minimum on step 6 with function value 5050.49
basinhopping step 7: f 5050.49 trial_f 5050.49 accepted 1  lowest_f 5050.49
basinhopping step 8: f 5050.49 trial_f 5050.49 accepted 1  lowest_f 5050.49
basinhopping step 9: f 5050.49 trial_f 5050.49 accepted 1  lowest_f 5050.49
found new global minimum on step 

C:\Users\Manik.Rajora\AppData\Local\Continuum\anaconda2\envs\Py36\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in true_divide
C:\Users\Manik.Rajora\AppData\Local\Continuum\anaconda2\envs\Py36\lib\site-packages\ipykernel_launcher.py:34: RuntimeWarning: divide by zero encountered in log
C:\Users\Manik.Rajora\AppData\Local\Continuum\anaconda2\envs\Py36\lib\site-packages\ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in multiply
C:\Users\Manik.Rajora\AppData\Local\Continuum\anaconda2\envs\Py36\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in true_divide
C:\Users\Manik.Rajora\AppData\Local\Continuum\anaconda2\envs\Py36\lib\site-packages\ipykernel_launcher.py:34: RuntimeWarning: divide by zero encountered in log
C:\Users\Manik.Rajora\AppData\Local\Continuum\anaconda2\envs\Py36\lib\site-packages\ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in multiply
C:\Users\Manik.Raj

basinhopping step 24: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 25: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 26: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 27: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 28: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 29: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 30: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 31: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 32: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 33: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 34: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 35: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 36: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 37: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 38: f nan trial_f nan accepted

basinhopping step 93: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 94: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 95: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 96: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 97: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 98: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 99: f nan trial_f nan accepted 1  lowest_f 5044.5
adaptive stepsize: acceptance rate 0.940000 target 0.500000 new stepsize 0.617284 old stepsize 0.555556
basinhopping step 100: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 101: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 102: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 103: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 104: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 105: f nan trial_f nan accepted 1  lowest_f 5044.5
basinh

basinhopping step 161: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 162: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 163: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 164: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 165: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 166: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 167: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 168: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 169: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 170: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 171: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 172: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 173: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 174: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 175: f nan trial

...

basinhopping step 433: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 434: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 435: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 436: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 437: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 438: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 439: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 440: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 441: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 442: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 443: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 444: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 445: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 446: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 447: f nan trial

basinhopping step 501: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 502: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 503: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 504: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 505: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 506: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 507: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 508: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 509: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 510: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 511: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 512: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 513: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 514: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 515: f nan trial

basinhopping step 569: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 570: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 571: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 572: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 573: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 574: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 575: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 576: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 577: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 578: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 579: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 580: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 581: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 582: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 583: f nan trial

basinhopping step 637: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 638: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 639: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 640: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 641: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 642: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 643: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 644: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 645: f nan trial_f nan accepted 1  lowest_f 5044.5
basinhopping step 646: f nan trial_f nan accepted 1  lowest_f 5044.5


KeyboardInterrupt: 

In [ ]:
print(res[0][0])


In [152]:
x = res[0][0]

def nn_objective_test(x):
    num_inputs = 11
    num_outputs = 1

    hidden_layer_neurons = 5
    
    weights_all = num_inputs*hidden_layer_neurons + hidden_layer_neurons*num_outputs
    bias_all = hidden_layer_neurons+num_outputs
    iw_size = num_inputs*hidden_layer_neurons
    nn_weights = x[:weights_all]
    bias_all = x[weights_all:]
    iw = nn_weights[0:iw_size]
    ow = nn_weights[iw_size:]
    b1 = bias_all[:hidden_layer_neurons]
    b2 = bias_all[hidden_layer_neurons:]
    w1 = np.reshape(iw,(num_inputs, hidden_layer_neurons))
    w2 = np.reshape(ow,(hidden_layer_neurons, num_outputs))
    l1 = 1/(1+np.exp(-(np.dot(testX,w1)+b1)))
#     l1 = np.maximum(np.dot(trainX, w1)+b1, 0)
    x_i = np.dot(l1, w2)+b2
#     l2 = np.array(np.exp(x_i)/ sum(np.exp(x_i)))
    l2 = np.exp(x_i) / (1 + np.exp(x_i))
    l2_act = np.round(l2*6+3)
    predict_out = l2_act
    predict_out = predict_out.flatten()
    acc = 1-accuracy_score(testY_comp,predict_out)
    print(testY_comp)
    print(predict_out)
    return acc

test_miss = nn_objective_test(x)
print(test_miss)

[ 6.  6.  7.  7.  6.  6.  6.  6.  5.  6.  6.  6.  6.  7.  6.  5.  6.  6.
  7.  5.  6.  6.  7.  7.  5.  5.  6.  7.  3.  5.  5.  5.  7.  6.  5.  6.
  5.  5.  5.  6.  6.  6.  6.  5.  6.  5.  7.  6.  5.  5.  5.  8.  7.  6.
  6.  6.  5.  7.  6.  6.  7.  7.  6.  7.  8.  9.  5.  6.  5.  7.  6.  6.
  7.  5.  5.  7.  6.  6.  6.  5.  7.  6.  6.  6.  6.  6.  6.  4.  4.  6.
  6.  6.  6.  8.  6.  4.  7.  5.  4.  6.  6.  5.  6.  7.  5.  7.  5.  5.
  6.  6.  7.  5.  6.  6.  7.  4.  6.  6.  6.  5.  6.  5.  6.  5.  7.  5.
  5.  6.  5.  5.  5.  5.  6.  6.  5.  5.  5.  6.  6.  5.  6.  7.  8.  6.
  5.  7.  5.  5.  8.  6.  6.  5.  6.  5.  7.  6.  7.  5.  6.  5.  6.  6.
  6.  6.  6.  6.  5.  6.  6.  6.  8.  6.  6.  5.  5.  4.  7.  5.  7.  6.
  5.  5.  7.  6.  8.  6.  7.  6.  5.  6.  6.  6.  6.  5.  5.  5.  6.  5.
  6.  6.  6.  4.  5.  6.  6.  5.  5.  6.  6.  5.  8.  5.  5.  5.  7.  5.
  6.  5.  5.  6.  6.  6.  6.  5.  8.  8.  5.  7.  7.  6.  7.  5.  7.  6.
  5.  5.  7.  6.  6.  5.  7.  5.  6.  5.  7.  7.  6

In [ ]:
import h5py
file_name = "weights.best_leaf.hdf5"
data = h5py.File(file_name,'r')

for key in data.keys():
    print(key) #Names of the groups in HDF5 file.

# group = data['model_weights']
# data = group['model_weights']

d = list(data['model_weights'])
print(d[:])

In [ ]:
from keras.models import load_model
model = load_model("weights.best_leaf.hdf5")
print(model.keys())

In [ ]:
import h5py
f = h5py.File("weights.best_leaf.hdf5", 'r')
print(list(f.keys()))
#will get a list of layer names which you can use as index
d = f['model_weights']['dense_1']
print(list(d.get('dense_1').keys()))
d.get('dense_1').get('bias:0')[()]